In [47]:
import numpy as np
import scipy.io as io
import scipy.io.wavfile as wav
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import tests
import scipy.stats
import time
from python_speech_features import mfcc

class randomD:
    distribution = []
    def __init__(self, array):
        self.distribution = array/np.sum(array)
        return
    def random(self, num):
        if num <= 0:
            print("please submit a positive integer")
            return
        return np.random.choice(range(self.distribution.shape[0]), num, p=self.distribution)
    def rand(self):
        return np.random.choice(range(self.distribution.shape[0]), 1, p=self.distribution)[0]
    def likelihood(self, num):
        if num>=self.distribution.shape[0] or num<0:
            print("emission out of bounds for distribution")
            return
        return self.distribution[num]
    def loghood(self, num):
        return np.log(self.likelihood(num))
    def dist(self):
        return self.distribution
    
class markovChain:
    A = None
    pi = None
    finite = False
    length = 0
    def __init__(self, initialPi, matrixA, finite = False):
        if matrixA.shape[0] is not matrixA.shape[1]:
            if matrixA.shape[0] == matrixA.shape[1] -1:
                self.finite = True
            else:
                print("error, your transition matrix was not symmetric")
                return
        if matrixA.shape[0] is not initialPi.shape[0]:
            print("error, the number of initial states does not match the transition matrix")
            return
        matrixA = np.array([i/np.sum(i) for i in matrixA])
        self.A = matrixA
        initialPi = initialPi/np.sum(initialPi)
        self.pi = initialPi
        return
    def random(self, num):
        if num <= 0:
            print("please submit a positive integer")
            return
        if self.finite:
            res = np.zeros(1, dtype=int)
            res[0] = np.random.choice(range(self.pi.shape[0]), 1, p=self.pi)
            num -= 1
            if num == 0:
                return res
            for i in range(num):
                temp = np.random.choice(range(self.A.shape[1]), 1, p=self.A[res[i]])
                if temp[0] >= self.A.shape[0]:
                    return res
                else:
                    res = np.append(res, temp)
            return res
        else:
            res = np.zeros((num), dtype = int)
            res[0] = np.random.choice(range(self.pi.shape[0]), 1, p=self.pi)
            num -= 1
            if num == 0:
                return res
            for i in range(num):
                res[i+1] = np.random.choice(range(self.A.shape[1]), 1, p=self.A[res[i]])    
        return res

    
class HMM:
    A = None
    B = None
    pi = None
    mc = None
    finite = False
    length = 0
    def __init__(self, initialPi, matrixA, matrixB, finite = False , length = 0, mc=None):
        if mc is not None:
            self.A = mc.A
            self.pi = mc.pi
            self.mc = mc
            self.finite = mc.finite
            self.B = matrixB
            self.length = length
            return
        else:
            if matrixA.shape[0] is not matrixA.shape[1]:
                print("error, your transition matrix was not symmetric")
                return
            if matrixA.shape[0] is not initialPi.shape[0]:
                print("error, the number of initial states does not match the transition matrix")
                return
            if matrixA.shape[0] is not matrixB.shape[0]:
                print("error, the number of states in the emission matrix does not match the transition matrix")
                return
            matrixA = np.array([i/np.sum(i) for i in matrixA])
            self.A = matrixA
            initialPi = initialPi/np.sum(initialPi)
            self.pi = initialPi
        self.B = matrixB
        self.finite = finite
        self.length = length
        return
    def random(self, num):
        if num <= 0:
            print("error, please submit a positive integer")
            return
        if self.finite and num > self.length:
            print("finite duration, only", self.length, "samples can be generated")
            return self.random(self.length)
        res = (np.zeros((num, self.B[0].random(1).ndim)), np.zeros((num), dtype = int))
        temp = np.random.choice(range(self.pi.shape[0]), 1, p=self.pi)[0]
        res[1][0] = temp
        res[0][0] = self.B[temp].rand()
        num -= 1
        if num == 0:
            return res
        for i in range(num):
            temp = np.random.choice(range(self.A.shape[0]), 1, p=self.A[res[1][i]])[0] 
            res[1][i+1] = temp
            dist = self.B[temp]
            res[0][i+1] = self.B[temp].rand()
        return res
    def alpha(self, obs):
        alpha = np.zeros((obs.shape[0], self.A.shape[0]))
        temp = np.zeros(self.A.shape[0])
        for i in range(temp.shape[0]):
            temp[i] = self.B[i].likelihood(obs[0])
        alpha[0, :] = self.pi * temp
        
        for t in range(1, obs.shape[0]):
            for j in range(self.A.shape[0]):
                alpha[t, j] = alpha[t - 1].dot(self.A[:, j]) * self.B[j].likelihood(obs[t])
        
        return alpha
    
    #this will probably at some point become the defactor alpha function, but while developing and testing
    #the loglikelihood version of a HMM (needed for large models) is separated
    def logalpha(self, obs):
        alpha = np.zeros((obs.shape[0], self.A.shape[0]))
        loga = np.log(self.A)
        logpi = np.log(self.pi)
        temp = np.zeros(loga.shape[0])
        for i in range(temp.shape[0]):
            temp[i] = self.B[i].loghood(obs[0])
        alpha[0] = logpi + temp
        
        for t in range(1, obs.shape[0]):
            for j in range(loga.shape[0]):
                for i in range(loga.shape[0]):
                    temp[i] = alpha[t-1, i] + loga[i,j] + self.B[j].loghood(obs[t])
                alpha[t, j] = lse(temp)
        return alpha
    
    def alphahat(self, obs, scale = True):
        if self.finite:
            A = self.A[:,:-1]
        else:
            A = self.A
        p, scaled = prob(obs, self.B)
        if not scale:
            scaled = p
        c = np.zeros(obs.shape[0])
        alpha = np.zeros((obs.shape[0], A.shape[0]))
        temp = np.zeros(A.shape[0])
        c[0] = np.sum(self.pi*scaled[0])
        alpha[0,:] = (self.pi*scaled[0])/c[0]

        for t in range(1, obs.shape[0]):
            for j in range(A.shape[0]):
                temp[j] = alpha[t-1].dot(A[:,j]) * scaled[t, j]
            c[t] = np.sum(temp)
            alpha[t, :] = temp/c[t]

        if self.finite:
            variable = alpha[-1].dot(self.A[:, -1])
            c = np.append(c, np.array([variable]))
        return alpha, c
    
    def joint(self, obs):
        alpha, c = self.alphahat(obs, scale = False)
        logp = np.log(c)
        return np.sum(logp)
            
    def beta(self, obs):
        beta = np.zeros((obs.shape[0], self.A.shape[0]))
        beta[obs.shape[0] - 1] = np.ones((self.A.shape[0]))
        
        for t in range(obs.shape[0] - 2, -1, -1):
            for j in range(self.A.shape[0]):
                temp = np.zeros(self.A.shape[0])
                for i in range(temp.shape[0]):
                    temp[i] = self.B[i].likelihood(obs[t+1])
                intermediate = (beta[t + 1] * temp).dot(self.A[j, :])
                beta[t, j] = intermediate
        return beta
    
    #same here - this is the future version of beta
    def logbeta(self, obs):
        beta = np.zeros((obs.shape[0], self.A.shape[0]))
        loga = np.log(self.A)
        logpi = np.log(self.pi)
        temp = np.zeros(loga.shape[0])
        
        for t in range(obs.shape[0]-2, -1, -1):
            for j in range(loga.shape[0]):
                for i in range(loga.shape[0]):
                    temp[i] = beta[t+1, i] + loga[j, i] + self.B[i].loghood(obs[t+1])
                beta[t, j] = lse(temp)
        return beta
    
    def betahat(self, obs, scale = True):
        if self.finite:
            A = self.A[:,:-1]
        else:
            A = self.A
        p, scaled = prob(obs, self.B)
        if not scale:
            scaled = p
        alphas, cs = self.alphahat(obs)
        beta = np.zeros((obs.shape[0], self.A.shape[0]))
        temp = np.zeros(self.A.shape[0])
        if self.finite:
            temp = self.A[:,-1]
            #print(cs[-1]*cs[-2])
            temp = temp/(cs[-1]*cs[-2])
        else:
            temp = np.ones((self.A.shape[0]))
            temp = temp/cs[-1]
        beta[-1] = temp
        
        for t in range(obs.shape[0]-2, -1, -1):
            temp = np.zeros(A.shape[0])
            for i in range(A.shape[0]):                
                for j in range(A.shape[0]):
                    temp[i] += A[i,j]*scaled[t+1,j]*beta[t+1, j]
            beta[t] = temp/cs[t]
        return beta
    
    def viterbi(self, obs, scale = True):
        if self.finite:
            A = self.A[:,:-1]
        else:
            A = self.A
        chi = np.zeros((obs.shape[0], A.shape[0]))
        prev = np.zeros((obs.shape[0]-1, A.shape[0]), dtype=int)
        p, scaled = logprob(obs, self.B)
        if not scale:
            scaled = p
        chi[0,:] = np.log(self.pi) + scaled[0]

        for t in range(1, obs.shape[0]):
            for j in range(A.shape[0]):
                #proba = chi[t-1] + np.log(A[:, j]) + scaled[t,j]
                #prev[t-1, j] = np.argmax(proba)
                #chi[t,j] = np.amax(proba)
                proba = chi[t-1] + np.log(A[:, j])             
                prev[t-1, j] = np.argmax(proba)
                chi[t,j] = np.amax(proba + scaled[t,j])
        
        if self.finite:
            chi[-1] += np.log(self.A[:, -1])
        ihat = np.zeros(obs.shape[0], dtype = int)
        last = np.argmax(chi[-1, :])
        ihat[0] = last
        
        index = 1
        for i in range(obs.shape[0]-2, -1, -1):
            temp = prev[i, int(last)]
            ihat[index] = temp
            last = temp
            index += 1
        ihat = np.flip(ihat, axis = 0)
        return ihat
    
    #page 113
    def calcgammas(self, alphahats, betahats, cs, obs, uselog=False):
            gammas = []
            for i in range(obs.shape[0]):
                temp = []
                for t in range(obs[i].shape[0]):
                    if uselog:
                        temp += [np.log(alphahats[i][t])+np.log(betahats[i][t])+np.log(cs[i][t])]
                    else:
                        temp += [alphahats[i][t]*betahats[i][t]*cs[i][t]]
                gammas += [np.array(temp)]
            gammas = np.array(gammas)
            return gammas
    
    #page 130
    def calcinit(self, gammas, uselog=False):
        if uselog:
            return np.sum(np.exp(gammas[:,0]), axis = 0)/np.sum(np.exp(gammas[:,0]))
        else: 
            return np.sum(gammas[:,0], axis = 0)/np.sum(gammas[:,0])
    
    #assignment 3 and 4. We got these correct...
    def calcabc(self, obs):
        alphahats = []
        betahats = []
        cs = []
        for i in range(obs.shape[0]):
            alph, c = self.alphahat(obs[i])
            beth = self.betahat(obs[i])
            alphahats += [alph]
            betahats += [beth]
            cs += [c]
        return alphahats, betahats, cs
    
    #page 132
    def calcxi(self, alphahats, betahats, cs, obs, uselog=False):
        xirbars = []
        xirs = []
        for i in range(obs.shape[0]): 
            if self.finite:
                xi = np.zeros((obs[i].shape[0], self.A.shape[0], self.A.shape[1]))
            else:
                xi = np.zeros((obs[i].shape[0]-1, self.A.shape[0], self.A.shape[1]))
            p, scaled = prob(obs[i], self.B)
            if uselog: 
                xi = np.log(xi)
                p, scaled = logprob(obs[i], self.B)
            for t in range(obs[i].shape[0]-1):
                for j in range(self.A.shape[0]):
                    for k in range(self.A.shape[0]):
                        if uselog:
                            xi[t, j, k] = np.log(alphahats[i][t][j])+np.log(self.A[j,k])+scaled[t+1][k]+np.log(betahats[i][t+1][k])
                        else:
                            xi[t, j, k] = alphahats[i][t][j]*self.A[j,k]*scaled[t+1][k]*betahats[i][t+1][k]
            if self.finite:
                for j in range(self.A.shape[0]):
                    if uselog:
                        xi[-1][j][-1] = np.log(alphahats[i][-1][j])+np.log(betahats[i][-1][j])+np.log(cs[i][-1])
                    else:
                        xi[-1][j][-1] = alphahats[i][-1][j]*betahats[i][-1][j]*cs[i][-1]
                
            if uselog:
                xi = np.exp(xi)
            xirs += [xi]
            xirbars += [np.sum(xi, axis = 0)]
        xibar = np.sum(xirbars, axis = 0)
        return xibar
    
    def printoutput(self, newmean, newcov):
        print("new initial:")
        print(self.pi)
        print()
        print("new A:")
        print(self.A)
        print()
        print("new means:")
        print(newmean)
        print()
        print("new covariances:")
        print(newcov)        
        
    
    def BW(self, obs, niter, uselog=False, prin=0, scaled = True):     
        A = self.A
        B = self.B
        for it in range(niter):
            alphahats, betahats, cs = self.calcabc(obs) #from Assignment 3 and 4
            gammas = self.calcgammas(alphahats, betahats, cs, obs, uselog) #alpha*beta*c
            newpi = self.calcinit(gammas, uselog) #average of gammas[:,0]
            xibar = self.calcxi(alphahats, betahats, cs, obs, uselog) #page 132
            if uselog: xibar = np.exp(xibar)
            newA = np.array([i/np.sum(i) for i in xibar]) #xibar/sum_k(xibar); page 130
            if uselog: gammas = np.exp(gammas)
            '''
            print("alphahats[0]")
            print(alphahats[0].shape)
            print(alphahats[0])
            print()
            print("betahats[0]")
            print(betahats[0].shape)
            print(betahats[0])
            print()
            print("cs[0]")
            print(cs[0].shape)
            print(cs[0])
            print()
            print("gammas")
            print(np.exp(gammas[:,-1]).shape)
            print(np.exp(gammas[:,-1]))
            print()
            print("xis")
            print(xibar.shape)
            print(xibar)
            print()
            '''
            
            #the above should be all that is needed for transition matrix and pi
            #below is fitting the output gaussians to a weighted average of the data
            #the weights for the average are the expected number of times being in a state
            
            #if uselog:
            #    for i in range(obs.shape[0]):
            #        loggammaobs = np.zeros(obs[i].shape[0], self.B.shape[0], obs[i].shape[1])
            #        loggammase = np.zeros(obs[i].shape[0], self.B.shape[0], obs[i].shape[1], obs[0].shape[1])
            #        for t in range(obs[i].shape[0]):
            #            for j in range(self.B.shape[0]):
            #                loggammaobs[t, j] = np.log(obs[i][t]) + gammas[i][t][j]
            #                temp = obs[i][t]-np.atleast_2d(self.B[j].getmean())
            #                for ii in range(obs[0].shape[1]):
            #                    loggammase[t,j, ii, ii] = gammas[i][t][j] + temp 
            #else:
            summ = np.zeros((self.B.shape[0], obs[0].shape[1]))
            sumc = np.zeros((self.B.shape[0], obs[0].shape[1], obs[0].shape[1]))
            sumg = np.zeros((self.B.shape[0]))

            for i in range(obs.shape[0]):
                for t in range(obs[i].shape[0]): 
                    for j in range(self.B.shape[0]):
                        summ[j] += obs[i][t]*gammas[i][t][j]
                        sumg[j] += gammas[i][t][j]
                        temp = obs[i][t] - np.atleast_2d(self.B[j].getmean())
                        sumc[j] += gammas[i][t][j]*(temp.T.dot(temp))
            newmean = np.zeros(summ.shape)
            newcov = np.zeros(sumc.shape)
            for i in range(newmean.shape[0]):
                newmean[i] = summ[i]/sumg[i]
                newcov[i] = sumc[i]/sumg[i]
            
            #if we get 0/0 then it becomes NaN. This sets it to 0.
            newpi[np.isnan(newpi)]=0
            newA[np.isnan(newA)]=0
            newmean[np.isnan(newmean)]=0
            
            #update all variables
            self.pi = newpi
            self.A = newA
            newB = np.array([multigaussD(newmean[i], newcov[i]) for i in range(B.shape[0])])
            self.B = newB
        if prin:    
            self.printoutput(newmean, newcov)
        return 
    
        
class gaussD:
    mean = 0
    std = 1
    def __init__(self, mu, sigmas):
        self.mean = mu
        self.std = abs(sigmas)
        return
    def random(self, num):
        return np.random.normal(self.mean, self.std, num)
    def rand(self):
        return np.random.normal(self.mean, self.std, 1)[0]
    def likelihood(self, num):
        mu = self.mean
        sigma = self.std
        tau = math.pi*2
        stau = math.sqrt(tau)
        fac1 = 1/(sigma*stau)
        expo = -0.5*(((num-mu)/sigma)**2)
        return fac1*(math.e**expo)
    def loghood(self, num):
        return np.log(self.likelihood(num))
    def getmean(self):
        return self.mean
    def getcov(self):
        return self.std
    
class multigaussD:
    mean = np.array([0])
    cov = np.array([[0]])
    def __init__(self, mu, C):
        if C.shape[0] is not C.shape[1]:
            print("error, non-square covariance matrix supplied")
            return
        if mu.shape[0] is not C.shape[0]:
            print("error, mismatched mean vector and covariance matrix dimensions")
            return
        self.mean = mu
        if np.where(np.diag(C)==0)[0].shape[0] != 0:
            C += np.diagflat(np.ones(C.shape[0])/10000)
        C[np.isnan(C)]=1
        self.cov = C
        return
    def random(self, num):
        return np.random.multivariate_normal(self.mean, self.cov, num)
    def rand(self):
        return np.random.multivariate_normal(self.mean, self.cov, 1)[0]
    def likelihood(self, X):
        p = scipy.stats.multivariate_normal(self.mean, self.cov, 1)
        pd = p.pdf(X)
        return pd
    def loghood(self, X):
        return np.log(self.likelihood(X))
    def getmean(self):
        return self.mean
    def getcov(self):
        return self.cov
    

def lse(arr):
    return np.log(np.sum(np.exp(arr)))

def prob(x, B):
    T = x.shape[0]
    N = B.shape[0]
    res = np.zeros((T, N))
    for i in range(T):
        for j in range(N):
            res[i,j] = B[j].likelihood(x[i])
    scaled = np.zeros(res.shape)
    for i in range(scaled.shape[0]):
        for j in range(scaled.shape[1]):
            scaled[i, j] = res[i,j]/np.amax(res[i])
    return res, scaled

def logprob(x, B):
    res, scaled = prob(x,B)
    return np.log(res), np.log(scaled)

def createA(length):
    A = np.zeros((length, length+1))
    for i in range(length):
        A[i,i] = 0.9
        A[i, i+1] = 0.1
    #A[-1,-1] = 1
    return A



In [50]:
####################################
# Using: 
# Miniconda3 with environment:
# Python 3.6.9
# Numpy 1.17.3
####################################

array = np.array([6.05379339e-08, 7.33819029e-36])
print("Array:")
print(array)
print("Sum of array:")
print(np.sum(np.array([6.05379339e-08, 7.33819029e-36])))
print("Using above listed python and numpy, this sum should be 6.05379339e-08")
print("Meaning it is the same as element 0")


Array:
[6.05379339e-08 7.33819029e-36]
Sum of array:
6.05379339e-08
Using above listed python and numpy, this sum should be 6.05379339e-08
Meaning it is the same as element 0


In [ ]:
a = time.time()        

pi = np.array([1, 0.0, 0.0, 0.0])
A = np.array([[0.95, 0.05, 0.0, 0.0],[0.0, 0.95, 0.05, 0.0], [0.0, 0.0, 0.95, 0.05], [0.0, 0.0, 0.0, 1.0]])
mean = np.array([0, 0])
cov = np.array([[1, 2], [2, 4]])
m = multigaussD(mean, cov)
mean2 = np.array([2, 2])
cov2 = np.array([[1, 0], [0,3]])
m2 = multigaussD(mean2, cov2)

m1star = np.array([4,4])
m2star = np.array([8,8])
cov1star = np.array([[1,0],[0,1]])
cov2star = np.array([[2,1],[1,2]])
g1 = multigaussD(m1star, cov1star)
g2 = multigaussD(m2star, cov2star)

Bstar = np.array([m, m2, g1, g2])

hm = HMM(pi, A, Bstar)

x1, real = hm.random(100)
x2, real2 = hm.random(100)
x3, real3 = hm.random(100)
x4, real4 = hm.random(100)
x5, real5 = hm.random(100)
x6, real6 = hm.random(100)
x7, real7 = hm.random(100)
x8, real8 = hm.random(100)
x9, real9 = hm.random(100)
x10, real10 = hm.random(100)
x11, real11 = hm.random(100)
x12, real12 = hm.random(100)
#print("real:\n",real)
#vit = hm.viterbi(x1)
#print("pred:\n", vit)

x = np.array([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12])

pistar = np.array([1, 0.0, 0.0, 0.0])
Astar = np.array([[0.8, 0.2, 0.0, 0.0], [0.0, 0.8, 0.2, 0.0], [0.0, 0.0, 0.8, 0.2], [0.0, 0.0, 0.0, 1.0]])
m1star = np.array([-1,0])
m2star = np.array([2,1])
cov1star = np.array([[1,1],[1,1]])
cov2star = np.array([[1,0],[0, 1]])
m1starr = np.array([3,3])
m2starr = np.array([5,6])
cov1starr = np.array([[1,0],[0,1]])
cov2starr = np.array([[2,1],[1, 2]])
g1 = multigaussD(m1star, cov1star)
g2 = multigaussD(m2star, cov2star)
g3 = multigaussD(m1starr, cov1starr)
g4 = multigaussD(m2starr, cov2starr)
B = np.array([g1, g2, g3, g4])

hm2 = HMM(pistar, Astar, B)

mc = markovChain(pi, np.array([[0.9, 0.1, 0.0, 0.0, 0.0],[0.0, 0.9, 0.1, 0.0, 0.0], [0.0, 0.0, 0.9, 0.1, 0.0], [0.0, 0.0, 0.0, 0.9, 0.1]]) , finite = False)

hm3 = HMM(pi, A, B, mc=mc)

print("BW:")
print(hm3.BW(x, 3, prin=1))

#print("\n\n")

#print(hm3.viterbi(x3))
#print(real3)

#prob, scaled = prob(x[0], Bstar)

#print(prob)

#print(hm.alphahat(x1[:10]))

#print(time.time()-a)

In [ ]:
r, fem = wav.read("female.wav")
fmfcc = mfcc(fem, r, nfft = 2048)
data = np.array([fmfcc, fmfcc])

for i in range(data.shape[0]):
    data[i] = (data[i]-np.mean(data[i]))/np.std(data[i])

#num regions: 22
covars = np.ones((22, 13, 13))
data2 = np.load("mel.npy", allow_pickle=True)
means = np.load("means.npy", allow_pickle=True)
B = np.array([multigaussD(means[i], covars[i]) for i in range(means.shape[0])])
A = createA(22)
pi = np.zeros(22)
pi[0] = 1
mc = markovChain(pi, A)
hm = HMM(pi, A, B, mc=mc)
#hm.BW(data, 1, prin=0)
#hm.BW(data, 1, prin=0)
#print("done")

#print(logprob(data[0], B)[1][0])

'''
hm.BW(np.array([data2[40]]), 1,  prin=1)
res = hm.viterbi(data2[41])
print(res)
hm.BW(np.array([data2[41]]), 1,  prin=0)
res = hm.viterbi(data2[42])
print(res)
hm.BW(np.array([data2[42]]), 1,  prin=0)
res = hm.viterbi(data2[43])
#res = hm.viterbi(fmfcc)
print(res)

#res = hm.viterbi(data2[41])
#print(res)

#bla = data2[0][0:40]
#print(np.mean(bla, axis = 0))
#print(np.mean(data[0][0:10], axis = 0))
#print(data2[42])

temp = (data[0]-np.mean(data[0]))/np.std(data[0])
from matplotlib import cm
plt.imshow(temp.T, interpolation='nearest', cmap=cm.coolwarm, origin='lower')
plt.show()
plt.imshow(data[0].T, interpolation='nearest', cmap=cm.coolwarm, origin='lower')
plt.show()
'''
obs = np.array([data[0]])
print(obs.shape)
uselog = False
alphahats, betahats, cs = hm.calcabc(obs) #from Assignment 3 and 4
gammas = hm.calcgammas(alphahats, betahats, cs, obs, uselog) #alpha*beta*c
print(gammas.shape)
newpi = hm.calcinit(gammas, uselog) #average of gammas[:,0]
print(newpi)
xibar = hm.calcxi(alphahats, betahats, cs, obs, uselog) #page 132
tempA = np.array([np.log(i) - np.log(np.sum(i)) for i in xibar])
print(tempA)

print(xibar[-1])

print("done")